In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from keras.datasets import mnist
import numpy as np
from network import Network

AttributeError: module 'pyarrow' has no attribute '__version__'

In [ ]:
def testing_network(datasets):
    for dataset_name in datasets:
        BASE_DIR = os.path.dirname(os.path.abspath(__file__))
        ruta_archivo = os.path.join(BASE_DIR, "../data/"+dataset_name)
        data = np.loadtxt(ruta_archivo, delimiter=',', dtype=int)
        X = data[:, :-1]
        y = data[:, -1] 

        dataset = [(X[i], [y[i]]) for i in range(len(X))]

        train_size = int(len(dataset) * 0.8)
        train_data = dataset[:train_size]
        test_data = dataset[train_size:]

        network1 = Network([2, 30, 1])
        network1.SGD(train_data, 10, 10, 0.5)
        accuracy, prediction = network1.evaluate(test_data)
        print('accuracy '+dataset_name )
        print(accuracy)

datasets = ['and_test.csv', 'or_test.csv', 'xor_test.csv']
# testing_network(datasets)


# ------------- training the mnist dataset -------------------
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28*28) / 255
X_test = X_test.reshape(-1, 28*28) / 255

def one_hot(y, n_classes=10):
    oh = np.zeros((y.size, n_classes))
    oh[np.arange(y.size), y] = 1
    return oh

y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)
training_data = list(zip(X_train, y_train_oh))
test_data = list(zip(X_test, y_test_oh))

net_mnist =  Network([784, 70, 70, 10], cost='cross entropy')
net_mnist.SGD(training_data, epochs=20, batch_size=25, lr=0.5)
accuracy, prediction = net_mnist.evaluate(test_data)
print('MNIST model Accuracy')
print(accuracy)

MNIST model Accuracy
0.9638
